In [17]:
import requests
from datetime import datetime, timedelta

TOKEN = ''

def get_all_logbook_entries(timestamp=None):
    url = f'http://ha.local:8123/api/logbook/2013-01-21T02:59:06.015463+00:00?end_time={datetime.now().isoformat()}'
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

logbook_entries = get_all_logbook_entries()

In [18]:
desired_log_domains = ["automation", "scene", "script", "switch", "light" ]
logbook_entries = [entry for entry in logbook_entries if entry.get('context_domain') in desired_log_domains]
logbook_entries

[{'when': '2024-07-08T00:46:25.845410+00:00',
  'state': 'on',
  'entity_id': 'light.emilys_bed_lamp',
  'name': 'Emily’s Bed Lamp',
  'context_user_id': '628cec48df964ae681e7c5a3b7461d59',
  'context_domain': 'scene',
  'context_service': 'turn_on',
  'context_event_type': 'call_service'},
 {'when': '2024-07-08T00:46:25.895722+00:00',
  'state': 'on',
  'entity_id': 'light.adrians_bed_lamp',
  'name': 'Adrian’s Bed Lamp',
  'context_user_id': '628cec48df964ae681e7c5a3b7461d59',
  'context_domain': 'scene',
  'context_service': 'turn_on',
  'context_event_type': 'call_service'},
 {'when': '2024-07-08T01:04:42.374077+00:00',
  'state': 'off',
  'entity_id': 'light.emilys_bed_lamp',
  'name': 'Emily’s Bed Lamp',
  'context_user_id': '628cec48df964ae681e7c5a3b7461d59',
  'context_domain': 'light',
  'context_service': 'turn_on',
  'context_event_type': 'call_service'},
 {'when': '2024-07-08T01:06:50.272158+00:00',
  'state': 'off',
  'entity_id': 'light.adrians_bed_lamp',
  'name': 'Adria

In [34]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

df = pd.DataFrame(logbook_entries)

def datetime_to_minutes_from_midnight(time_string):
    d = datetime.fromisoformat(time_string)
    return d.hour * 60 + d.minute * 1

df['time'] = df['when'].apply(lambda x: datetime_to_minutes_from_midnight(x))

ValueError: Shape of passed values is (13340, 1), indices imply (13340, 43)